In [4]:
import numpy as np
import numpy.ma as ma
import math

lambdaa = 80
lambdas = 200
taus = 200
taua = 200

def frameDevFunction(i, j, v):
    frameCostValue = min(math.pow(((j-i)-v),2), taus)
    return frameCostValue

def frameAccelCost(h, i, j):
    frameaccelerationcost = min(math.pow(((j-i) - (i-h)),2), taua)
    return frameaccelerationcost

def arrayPath(v):
    #v is targetted speed up; v<=extraDiag
    print "Loading Array"
    errors = np.loadtxt('swiss_dssim.txt')
    print "Array Loaded"
    frameCount = errors.shape[0]
    # extraDiag = create a mask of the top row and measure the length
    diagmaskedarray = ma.masked_equal(errors[0,:],0)
    extraDiag = int(diagmaskedarray.count()) -1
    #Add our speed up cost function
    startrange = 2
    for i in range(0, startrange):
        for j in range(i+1, i+extraDiag+2):
            if(j<frameCount and i<frameCount):
                errors[i,j] += lambdas * frameDevFunction(i, j, v)
    print "Start Gap Initialised"
    
    #Array is populated now assess costs
    costmatrix = np.zeros((frameCount, frameCount))
    #First Pass of Populate
    for i in range(startrange,frameCount):
        for j in range(i+1, i+extraDiag+2):
            if(j>=frameCount):
                continue
            Const = errors[i,j] + lambdas*frameDevFunction(i, j, v)
            n1 = []    
            #Populate a new list based on the for loop
            for k in range(1, extraDiag+2):
                #Populate our list
                n1.insert(k, (errors[(i-k), i] + lambdaa*frameAccelCost((i-k), i, j)))
            if(i< frameCount and j<frameCount):
                errors[i, j]= Const +  np.min(n1)            
                costmatrix[i, j] = i - (np.argmin(n1)+1)
                #Ensure that the line above is correct when we convert the index from n1
    print "Cost Matrix Calculated"
                                             
    #Second Pass Trace Back Min Cost
    #How many frames back are we willing to go for our final frame?
    endrange = 2
    #Create a masked array that masks zeros in the last few rows and columns of our costmatrix
    maskedarray = ma.masked_equal(costmatrix[(frameCount-(endrange+1)):, (frameCount-(endrange+1)):], 0)
    #The argmin from this new masked array are then used for the s and d values, unravel and the addition
    #Of the frame count puts them into the correct values with regards to the size of our original cost matrix
    s= int((np.unravel_index(maskedarray.argmin(), maskedarray.shape)[0]) + (frameCount-(endrange+1)))
    d= int((np.unravel_index(maskedarray.argmin(), maskedarray.shape)[1]) + (frameCount-(endrange+1)))
    p = []
    while(s>startrange):
        #Prepend to List and work our way back up through the cost matrix
        p.insert(0, int(s))
        b = costmatrix[s, d]
        d = int(s)
        s = int(b)
    np.savetxt('framelist_swiss_dssim_5x_80_200_200_200.txt',p)
    print "Frame Path Saved"

arrayPath(5)


Loading Array
Array Loaded
Start Gap Initialised
Cost Matrix Calculated
Frame Path Saved
